RUN003 - Run expert rules
=========================

Description
-----------

Run follow-on notebooks (i.e. SOPs/TSGs) where ‘expert rule’ expressions
defined in the notebook `notebook_path` match the output in the
notebook. (this presumes the notebook defined in `notebook_path` is an
executed notebook, with ‘outputs’ cells, and usually has a filename
prefix of ‘output-’).

Example ‘expert rules’:

1.  To match text in an exception:

    \[“TST008a”, “code”, “error”, “ename”, “SystemExit”, “evalue”,
    ".\*Match this text"\]

2.  To match text in a normal stdout:

    \[“TST008b”, “code”, “stream”, “name”, “stdout”, “text”, ".\*This is
    the expression to match"\]

The “code”, “stream”, “name”, “stdout”, values of the elements in the
rule definition above are matched against the cells list in the
“output-” notebook .ipynb file. The definition of each element is:

1.  The notebook identifier to run if expression is matched,
    i.e. “TST008a”
2.  The cell\_type to match i.e. “code”
3.  The output\_type to match i.e. “error” or “stream”
4.  The output\_type\_name to match i.e. “ename” or “name”
5.  The name of the output type to match i.e. “SystemExit” or “stdout”
6.  The element containing the expression to match i.e. “evalue or”text"
7.  The regular expression to match, i.e. “.*Match this text" or ".*This
    is the expression to match”
8.  (optional) A priority for the rule (0 == highest priority)

In the output-\* notebook .ipynb, this is what the structure of the JSON
is that will match these two rule examples:

1.  The output to match text in an exception:

<!-- -->

      {
       "cell_type": "code",
       },
       "outputs": [
       {
         "ename": "SystemExit",
         "evalue": "Match this text",
         "output_type": "error",
         ]
       }

1.  The output to match text in a stdout:

<!-- -->

      {
       "cell_type": "code",
       },
       "outputs": [
        {
         "name": "stdout",
         "output_type": "stream",
         "text": [
          "This is the expression to match\n"
         ]
        }

Expert rules are stored in the metadata of a source notebook in
unexpanded form e.g.:

      "metadata": {
      "azdata": {
       "expert": {
        "rules": [
         [
          "RUN504a",
          "code",
          "stream",
          "name",
          "stdout",
          "text",
          ".\\*This is the expression to match"
         ]
        ]
       }
      }

And then injected in expanded form (identifier to filename expansion)
into the target notebook when the notebook (.ipynb) file is built by the
engineering system, e.g:

In this example, we see the source notebook above, RUN504b, has an
expert rule, that has been injected into RUN504b below. When the
expression ‘This is the expression to match’ is matched in the output of
a ‘stdout’ cell in RUN504a, the notebook RUN504b will be automatically
run.

        "metadata": {
            "azdata": {
                "expert": {
                    "expanded_rules": [
                        [
                            5,
                            "../notebook-runner/run504b-expert-rule.ipynb",
                            "code",
                            "stream",
                            "name",
                            "stdout",
                            "text",
                            ".\\*This is the expression to match",
                            0
                        ]
                    ]
                }
            }
        }

### Parameters

In [ ]:
import os
import getpass
import datetime

# The executed notebook to analyze
#
notebook_path = os.path.join(os.getcwd(), "output-run505a-sample-notebook.ipynb")

# In the app-deploy app folder, the .ipynbs are flattened into one folder, in the book, they are in folders.
#
if os.path.exists("run001-run-notebook.ipynb"):
    run001_full_path = "run001-run-notebook.ipynb"
else:
    run001_full_path = os.path.join("..", "notebook-runner", "run001-run-notebook.ipynb")

timeout = 600 # Per cell timeout in seconds (10 minutes)

app_name = "app-" + getpass.getuser().lower() # set default to be app-<username>
app_version = "v1"

session_start = str(datetime.datetime.utcnow())

step = "" # If this notebook is recursively called as an 'expert rule', this is the step we are on
base_notebook_name = "" # For expert rules, this is original calling notebook
parent_notebook_name = "" # For expert rules, this is the calling (parent) notebook

NOTEBOOK_CELL_TIMEOUT = 600 # Per cell timeout in seconds (10 minutes)

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, hyperlinked suggestions, and scrolling updates on Windows
import sys
import os
import re
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

def run(cmd, return_output=False, no_output=False, retry_count=0, base64_decode=False, return_as_json=False, regex_mask=None):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportability, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    # Display an install HINT, so the user can click on a SOP to install the missing binary
    #
    if which_binary == None:
        print(f"The path used to search for '{cmd_actual[0]}' was:")
        print(sys.path)

        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    cmd_display = cmd
    if regex_mask is not None:
        regex = re.compile(regex_mask)
        cmd_display = re.sub(regex, '******', cmd)
        
    print(f"START: {cmd_display} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    if base64_decode:
                                        import base64
                                        return base64.b64decode(output).decode('utf-8')
                                    else:
                                        return output

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        if base64_decode:
            import base64
            return base64.b64decode(output).decode('utf-8')
        else:
            return output



# Hints for tool retry (on transient fault), known errors and install guide
#
retry_hints = {'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use', 'Login timeout expired (0) (SQLDriverConnect)', 'SSPI Provider: No Kerberos credentials available',  ], 'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',  ], 'python': [ ], }
error_hints = {'azdata': [['Please run \'azdata login\' to first authenticate', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Can\'t open lib \'ODBC Driver 17 for SQL Server', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb'], ['NameError: name \'azdata_login_secret_name\' is not defined', 'SOP013 - Create secret for azdata login (inside cluster)', '../common/sop013-create-secret-for-azdata-login.ipynb'], ['ERROR: No credentials were supplied, or the credentials were unavailable or inaccessible.', 'TSG124 - \'No credentials were supplied\' error from azdata login', '../repair/tsg124-no-credentials-were-supplied.ipynb'], ['Please accept the license terms to use this product through', 'TSG126 - azdata fails with \'accept the license terms to use this product\'', '../repair/tsg126-accept-license-terms.ipynb'],  ], 'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb'],  ], 'python': [['Library not loaded: /usr/local/opt/unixodbc', 'SOP012 - Install unixodbc for Mac', '../install/sop012-brew-install-odbc-for-sql-server.ipynb'], ['WARNING: You are using pip version', 'SOP040 - Upgrade pip in ADS Python sandbox', '../install/sop040-upgrade-pip.ipynb'],  ], }
install_hint = {'azdata': [ 'SOP063 - Install azdata CLI (using package manager)', '../install/sop063-packman-install-azdata.ipynb' ],  'kubectl': [ 'SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb' ],  }


print('Common functions defined successfully.')

### Define helper functions

In [ ]:
# Define 'load_json' helper function

import json

def load_json(filename):
    with open(filename, encoding="utf8") as json_file:
        return json.load(json_file)

print ("'load_json' function defined")

### Run follow-on notebooks for each ‘expert rule’ expression match on output

If there are `expert rules` in this notebook, run the follow on
notebooks (i.e. TSGs)

In [ ]:
rules = []

# Open the output of this notebook to get the expressions/notebooks to match in the output cells
#
j = load_json(notebook_path)

if "metadata" in j and \
    "azdata" in j["metadata"] and \
    "expert" in j["metadata"]["azdata"] and \
    "expanded_rules" in j["metadata"]["azdata"]["expert"]:

    rules = j["metadata"]["azdata"]["expert"]["expanded_rules"]

    rules.sort() # Sort rules, so they run in priority order (the [0] element).  Lowest value first.

    print (f"EXPERT: There are {len(rules)} rules to evaluate.")
else:
    print('EXPERT: No expert rules to evaluate')

In [ ]:
import platform
from shutil import copyfile

for rule in rules:
    notebook = rule[1]
    cell_type = rule[2]
    output_type = rule[3] # i.e. stream or error
    output_type_name = rule[4] # i.e. ename or name 
    output_type_value = rule[5] # i.e. SystemExit or stdout
    details_name = rule[6]  # i.e. evalue or text 
    expression = rule[7].replace("\\*", "*") # Something escaped *, and put a \ in front of it!
    timeout = NOTEBOOK_CELL_TIMEOUT if rule[8] == 0 else rule[8]

    print(f"EXPERT: If rule '{expression}' satisfied', run '{notebook}'.")

    match_found = False # We only match once per rule (no need to run the same notebook (TSG/SOP) more than once!)
    for cell in j["cells"]:
        if cell["cell_type"] == cell_type:
            for output in cell["outputs"]:
                if output_type_name in output and output[output_type_name] == output_type_value:

                    print("EXPERT: output['{0}'] == output_type_value: '{1}' == '{2}' in '{3}'".format(
                        output_type_name, output[output_type_name], output_type_value, os.path.basename(notebook_path)))

                    # The 'error' type is a single value, convert to a list so we can process all types the same way
                    if output_type == "error":
                        details = [ output[details_name] ]
                    else: # stream
                        details = output[details_name]

                    for detail in details:
                        if re.match(expression, detail, re.DOTALL):
                            # Start the recursive process of running notebooks based on expert rules

                            print("EXPERT: MATCH: matched expression '{0}' in '{1}', therefore running '{2}'".format(
                                expression, os.path.basename(notebook_path), os.path.basename(notebook)))

                            # Special case for app-deploy, where notebooks are flattened into cwd directory structure
                            # 
                            if os.path.isfile(os.path.join(os.path.basename(notebook))):
                                notebook = os.path.join(os.path.basename(notebook))

                            # The file path in the expert rule is Linux based, i.e. '/', switch '\' on Windows
                            #
                            if platform.system() == "Windows":
                                notebook = notebook.replace("/", '\\')

                            match_found = True

                            args = {
                                "namespace": namespace, 
                                "session_start": session_start, 
                                "notebook_path": notebook.replace("\\", "\\\\").replace("\\", "\\\\"),
                                "step": step,
                                "app_name": app_name,
                                "app_version": app_version,
                                "base_notebook_name": base_notebook_name.replace(".ipynb", ""),
                                "parent_notebook_name": parent_notebook_name.replace(".ipynb", ""),
                                "save_results_in_storage_pool": save_results_in_storage_pool
                            }

                            args = str(args).replace("'", '\\"') # Windows cmd line, requires ", not '

                            # Create a copy of run001, so the results for each notebook are seperated
                            #
                            run001_copy_full_path = "run001-run-notebook-" + os.path.basename(notebook)

                            copyfile("run001-run-notebook.ipynb", run001_copy_full_path)

                            run('azdata notebook run --path {0} --output-html --output-path {1} --timeout {2} --arguments "{3}"'.format(
                                run001_copy_full_path,
                                os.getcwd().replace("\\", "\\\\"), 
                                timeout,
                                args))

                            os.remove(run001_copy_full_path)

                            break
                        else:
                            print("EXPERT: NON-MATCH: '{0}' not a regex match on '{1}'".format(
                                detail, expression))


                if not match_found:
                    if output_type_name in output:
                        print(f'EXPERT: Not a match on name: {output[output_type_name]} != {output_type_name}')
                    else:
                        print(f"EXPERT: Not a match, name not in output (therefore '{output_type_name}' not found).")
                else:
                    break
        else:
            print(f'EXPERT: Not a match on cell type: {cell["cell_type"]} != {cell_type}')
        
        if match_found:
            break

In [ ]:
print("Notebook execution is complete.")

Related
-------

- [RUN000 - Setup Master Pool runner infrastructure](../notebook-runner/run000-setup-infrastructure.ipynb)
- [RUN001 - Run a notebook](../notebook-runner/run001-run-notebook.ipynb)
- [RUN002 - Save result in Big Data Cluster](../notebook-runner/run002-save-result-in-bdc.ipynb)
